<a href="https://colab.research.google.com/github/akshatamadavi/data_mining/blob/main/unsloth_ai/03_rl_prefs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03_rl_prefs.ipynb — DPO (Preference Optimization) with Unsloth

**Goal:** Train a small model with **Direct Preference Optimization (DPO)** using a dataset that contains an input prompt and **preferred vs. rejected** responses. Designed for Google Colab (T4/A100 friendly).

**What this notebook covers**
1. Environment & GPU check
2. Install deps (`unsloth`, `trl`, `transformers`, `datasets`, `bitsandbytes`, `peft`)
3. Load a small Unsloth model (default: `unsloth/SmolLM2-135M-Instruct-bnb-4bit`)
4. Build a tiny **preference dataset** (or plug your own)
5. Train with **DPO** (LoRA by default for efficiency)
6. Save checkpoint + quick inference sanity-check

> Tip: Replace the toy dataset with your real dataset in the same schema (`prompt`, `chosen`, `rejected`).

In [1]:
#@title ⏱️ Setup — GPU check
import torch
print("Torch:", torch.__version__)
gpu = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"
print("GPU:", gpu)

Torch: 2.8.0+cu126
GPU: Tesla T4


In [2]:
#@title 📦 Install libraries (takes ~2–4 min on Colab)
!pip -q install -U unsloth trl transformers datasets bitsandbytes peft accelerate
import os
os.environ["BITSANDBYTES_NOWELCOME"] = "1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.3/351.3 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 20.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

In [3]:
#@title 🔧 Config — choose a base model & training knobs
from dataclasses import dataclass

BASE_MODEL = "unsloth/SmolLM2-135M-Instruct-bnb-4bit"  # small & fast; switch if needed
MAX_SEQ_LEN = 1024
BATCH_PER_DEVICE = 4
GRAD_ACCUM = 8
EPOCHS = 2  # keep small for demo; increase for real runs
LR = 2e-5
SEED = 42
OUTPUT_DIR = "outputs_dpo"
BETA = 0.1  # DPO strength

print({k:v for k,v in dict(BASE_MODEL=BASE_MODEL,MAX_SEQ_LEN=MAX_SEQ_LEN,BATCH_PER_DEVICE=BATCH_PER_DEVICE,GRAD_ACCUM=GRAD_ACCUM,EPOCHS=EPOCHS,LR=LR,SEED=SEED,OUTPUT_DIR=OUTPUT_DIR,BETA=BETA).items()})

{'BASE_MODEL': 'unsloth/SmolLM2-135M-Instruct-bnb-4bit', 'MAX_SEQ_LEN': 1024, 'BATCH_PER_DEVICE': 4, 'GRAD_ACCUM': 8, 'EPOCHS': 2, 'LR': 2e-05, 'SEED': 42, 'OUTPUT_DIR': 'outputs_dpo', 'BETA': 0.1}


## 1) Load model (Unsloth + 4-bit) and patch DPO
This:
- Loads a small instruct model in **4-bit** (saves VRAM)
- Adds **LoRA adapters** for efficient DPO
- Patches TRL's `DPOTrainer` with Unsloth kernels

In [4]:
from unsloth import FastLanguageModel, PatchDPOTrainer, is_bfloat16_supported
from transformers import TrainingArguments
from trl import DPOTrainer
import torch

PatchDPOTrainer()  # enable Unsloth's faster DPO

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL,
    max_seq_length = MAX_SEQ_LEN,
    dtype = None,
    load_in_4bit = True,
)

# Add fast LoRA adapters (PEFT)
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",  # memory saver
    random_state = SEED,
    max_seq_length = MAX_SEQ_LEN,
)

print("Model & tokenizer ready.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/112M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/423 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Unsloth 2025.11.2 patched 30 layers with 30 QKV layers, 30 O layers and 30 MLP layers.


Model & tokenizer ready.


## 2) Build a **toy preference dataset** (replace with your own)
Required columns:
- `prompt`: the instruction/input shown to the model
- `chosen`: the **preferred** response
- `rejected`: the **dispreferred** response

You can load from JSON/CSV too; just keep the same column names.

In [5]:
import pandas as pd
from datasets import Dataset

toy = [
    {
        "prompt": "You are a helpful coding assistant. Write a Python function to add two numbers with type hints.",
        "chosen": "def add(a: float, b: float) -> float:\n    return a + b",
        "rejected": "Use Java: int add(int a, int b) { return a + b; }",
    },
    {
        "prompt": "Explain the concept of overfitting in one short paragraph.",
        "chosen": "Overfitting happens when a model memorizes patterns specific to the training set and fails to generalize. It fits noise, leading to low training error but high validation error.",
        "rejected": "Overfitting is good because it reduces error everywhere and means the model is perfect.",
    },
    {
        "prompt": "Summarize why version control (git) is useful for data science projects.",
        "chosen": "Git tracks changes, enables collaboration via branches/PRs, makes experiments reproducible, and supports rollback, code review, and CI workflows.",
        "rejected": "Version control is unnecessary; just share code via email attachments.",
    },
]

df = pd.DataFrame(toy)
train_ds = Dataset.from_pandas(df)
train_ds

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 3
})

## 3) Train with **DPO**
For quick tests we keep small batch/epochs. Increase for stronger learning.

Notes:
- We pass **raw strings**; the trainer uses the tokenizer internally.
- `ref_model=None` uses an implicit reference (KL-free DPO variant).

In [38]:
args = TrainingArguments(
    per_device_train_batch_size = BATCH_PER_DEVICE,
    gradient_accumulation_steps = GRAD_ACCUM,
    warmup_ratio = 0.1,
    num_train_epochs = EPOCHS,
    learning_rate = LR,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 1,
    optim = "adamw_8bit",
    seed = SEED,
    output_dir = OUTPUT_DIR,
)

# Explicitly set tokenizer pad_token and padding_side
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if tokenizer.padding_side is None:
    tokenizer.padding_side = "right"

# Add padding_value attribute to args as expected by Unsloth's patched DPOTrainer
args.padding_value = tokenizer.pad_token_id

# Add model_init_kwargs to args as expected by Unsloth's patched DPOTrainer
args.model_init_kwargs = None

# Add ref_model_init_kwargs to args as expected by Unsloth's patched DPOTrainer
args.ref_model_init_kwargs = None

# Add generate_during_eval to args as expected by Unsloth's patched DPOTrainer
args.generate_during_eval = False

# Add model_adapter_name to args as expected by Unsloth's patched DPOTrainer
args.model_adapter_name = None

# Add ref_adapter_name to args as expected by Unsloth's patched DPOTrainer
args.ref_adapter_name = None

# Add reference_free to args as expected by Unsloth's patched DPOTrainer when ref_model is None
args.reference_free = True

# Add disable_dropout to args as expected by Unsloth's patched DPOTrainer
# Assuming False is a safe default unless specific dropout control is intended.
args.disable_dropout = False

# Add use_liger_loss to args as expected by Unsloth's patched DPOTrainer
args.use_liger_loss = False

# Add label_pad_token_id to args as expected by Unsloth's patched DPOTrainer
args.label_pad_token_id = tokenizer.pad_token_id

# Add max_prompt_length to args as expected by Unsloth's patched DPOTrainer
args.max_prompt_length = 512

# Add max_completion_length to args as expected by Unsloth's patched DPOTrainer
args.max_completion_length = MAX_SEQ_LEN - args.max_prompt_length

# Add max_length to args as expected by Unsloth's patched DPOTrainer
args.max_length = MAX_SEQ_LEN

# Add truncation_mode to args as expected by Unsloth's patched DPOTrainer
args.truncation_mode = None

# Add precompute_ref_log_probs to args as expected by Unsloth's patched DPOTrainer
args.precompute_ref_log_probs = False

# Add use_logits_to_keep to args as expected by Unsloth's patched DPOTrainer
args.use_logits_to_keep = False

# Add padding_free to args as expected by Unsloth's patched DPOTrainer
args.padding_free = False

# Add beta to args as expected by Unsloth's patched DPOTrainer
args.beta = BETA

# Add label_smoothing to args as expected by Unsloth's patched DPOTrainer
args.label_smoothing = 0.0 # Default value for label smoothing

# Add loss_type to args as expected by Unsloth's patched DPOTrainer
args.loss_type = "sigmoid" # Changed from "dpo" to "sigmoid" to match supported types

# Add loss_weights to args as expected by Unsloth's patched DPOTrainer
args.loss_weights = None

# Add use_weighting to args as expected by Unsloth's patched DPOTrainer
args.use_weighting = False

# Add f_divergence_type to args as expected by Unsloth's patched DPOTrainer
args.f_divergence_type = None

# Add f_alpha_divergence_coef to args as expected by Unsloth's patched DPOTrainer
args.f_alpha_divergence_coef = None

# Add dataset_num_proc to args as expected by Unsloth's patched DPOTrainer
args.dataset_num_proc = None

# Add tools to args as expected by Unsloth's patched DPOTrainer
args.tools = None

# Add sync_ref_model to args as expected by Unsloth's patched DPOTrainer
args.sync_ref_model = False

# Add rpo_alpha to args as expected by Unsloth's patched DPOTrainer
args.rpo_alpha = None

# Add ld_alpha to args as expected by Unsloth's patched DPOTrainer
args.ld_alpha = None

trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = args,
    beta = BETA,
    train_dataset = train_ds,
    tokenizer = tokenizer,
    max_length = MAX_SEQ_LEN,
)

trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

Extracting prompt in train dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3 | Num Epochs = 2 | Total steps = 2
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 9,768,960 of 144,284,544 (6.77% trained)


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
1,3.305700,0.000000,0.000000,0.000000,0.000000,-88.279984,-61.547142,8.351044,9.039118,0,0,0
2,3.305700,0.000000,0.000000,0.000000,0.000000,-88.279984,-61.547142,8.351044,9.039118,No Log,No Log,No Log


('outputs_dpo/tokenizer_config.json',
 'outputs_dpo/special_tokens_map.json',
 'outputs_dpo/chat_template.jinja',
 'outputs_dpo/vocab.json',
 'outputs_dpo/merges.txt',
 'outputs_dpo/added_tokens.json',
 'outputs_dpo/tokenizer.json')

## 4) Quick inference sanity check
We compare generations **before vs after** training on a held-out prompt.

In [39]:
from transformers import TextStreamer
import copy

def chat_once(prompt: str, max_new_tokens: int = 120):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.7, top_p=0.9)
    return tokenizer.decode(out[0], skip_special_tokens=True)

test_prompt = "Explain dropout in neural networks simply."
print("\n=== Sample Generation ===")
print(chat_once(test_prompt))


=== Sample Generation ===
Explain dropout in neural networks simply. This is because the network is trying to build a model of the world, which is what a neural network is. So it's just trying to build a model of the world, which is what a neural network is.

So you start thinking about the world, which is what a neural network is. So you start thinking about what's important, what's most important, what's most important, and so on. So that's what the model is. And then you start thinking about what's most important, and so on. So that's what a neural network is. So that's


## 5) (Optional) Load from the saved checkpoint later
Useful when resuming in a fresh Colab session.

In [ ]:
# # Example: Reload
# model2, tokenizer2 = FastLanguageModel.from_pretrained(
#     model_name = OUTPUT_DIR,
#     max_seq_length = MAX_SEQ_LEN,
#     dtype = None,
#     load_in_4bit = False,  # loading your LoRA-adapted weights
# )
# print("Reloaded.")

## 6) Swap to your dataset
Prepare a dataframe with **three columns**: `prompt`, `chosen`, `rejected`. Then create the Hugging Face `Dataset`.

```python
import pandas as pd
from datasets import Dataset

df = pd.read_json("/path/to/your_prefs.jsonl", lines=True)
train_ds = Dataset.from_pandas(df)
```

If your model uses a special **chat template**, you can embed your prompts accordingly, but DPO works fine with plain text prompts for many use cases.

---
### Notes
- This notebook uses **Unsloth's `PatchDPOTrainer`**, which wires up faster kernels under the hood and reduces VRAM.
- For bigger datasets, bump `EPOCHS`, consider gradient checkpointing (already on via `use_gradient_checkpointing="unsloth"`).
- If you see OOM errors, reduce `BATCH_PER_DEVICE` or `MAX_SEQ_LEN`.
- You can switch `BASE_MODEL` to any Unsloth-supported instruct model (e.g., Gemma/Llama/Mistral) that fits your GPU.

Happy training! 🦥